In [7]:
$env:AWS_PROFILE="mark"

In [ ]:
aws s3 ls

In [8]:
aws cognito-identity create-identity-pool `
    --identity-pool-name PollyIdentityPool `
    --allow-unauthenticated-identities `
    --region us-east-1 `
| Set-Variable IDENTITY_POOL_RESPONSE

Write-Host $IDENTITY_POOL_RESPONSE

# Copy the IdentityPoolId: from the output and assign it to the $IDENTITY_POOL_ID variable.


- AllowUnauthenticatedIdentities: true   IdentityPoolId: us-east-1:5ee4b58d-942a-4568-a4d5-4dcc551259c3   IdentityPoolName: PollyIdentityPool   IdentityPoolTags: {}


In [11]:
$IDENTITY_POOL_ID = "us-east-1:5ee4b58d-942a-4568-a4d5-4dcc551259c3"
Write-Host $IDENTITY_POOL_ID

# YOU MUST REPLCE "YOUR_IDENTITY_POOL_ID" in the trust-policy.json

us-east-1:5ee4b58d-942a-4568-a4d5-4dcc551259c3


In [29]:
Get-Content "trust-policy.json" `
| -Replace "YOUR_IDENTITY_POOL_ID","$IDENTITY_POOL_ID"
| Set-Content ".\trust-policy.json"

Get-Content: A parameter cannot be found that matches parameter name 'Replace'.


Error: Command failed: SubmitCode: Get-Content "trust-policy.json" -Replace "YOUR_IDE ...

In [12]:
aws iam create-role --role-name Cognito_PollyUnauth_Role --assume-role-policy-document file://trust-policy.json


- Role:
    Arn: arn:aws:iam::318364255844:role/Cognito_PollyUnauth_Role
    AssumeRolePolicyDocument:
      Statement:
      - Action: sts:AssumeRoleWithWebIdentity
        Condition:
          StringEquals:
            cognito-identity.amazonaws.com:aud: YOUR_IDENTITY_POOL_ID
        Effect: Allow
        Principal:
          Federated: cognito-identity.amazonaws.com
      Version: '2012-10-17'
    CreateDate: '2024-04-14T03:38:43+00:00'
    Path: /
    RoleId: AROAUUH75AZSGE6WE6JJR
    RoleName: Cognito_PollyUnauth_Role


In [13]:
aws iam attach-role-policy `
    --role-name Cognito_PollyUnauth_Role `
    --policy-arn "arn:aws:iam::aws:policy/AmazonPollyReadOnlyAccess"


In [17]:
aws sts get-caller-identity --query "Account" --output text `
| Set-Variable AWS_ACCOUNT_ID

Write-Host $AWS_ACCOUNT_ID


318364255844


In [24]:
aws cognito-identity list-identity-pools --max-results 60 --region us-east-1


- IdentityPools:
  - IdentityPoolId: us-east-1:43bb1378-1d99-4749-87bd-7e19a047871b
    IdentityPoolName: amplify_backend_manager_d208wyr2a86kkx
  - IdentityPoolId: us-east-1:5ee4b58d-942a-4568-a4d5-4dcc551259c3
    IdentityPoolName: PollyIdentityPool
  - IdentityPoolId: us-east-1:b6f7395c-e693-4f61-8f59-9f0896f8f45d
    IdentityPoolName: amplify_backend_manager_d2wf2k02o94nhk
  - IdentityPoolId: us-east-1:c6b7a0d6-d027-49ea-b061-2135a9afce31
    IdentityPoolName: mygrandparentsc94844b7_identitypool_c94844b7__staging
  - IdentityPoolId: us-east-1:d53bd38d-3d3d-47d1-b5fd-9b7d956ac319
    IdentityPoolName: amplify_backend_manager_djgfn2g00aoen


In [30]:
aws cognito-identity set-identity-pool-roles `
    --profile mark `
    --identity-pool-id $IDENTITY_POOL_ID `
    --region us-east-1 `
    --roles "unauthenticated=arn:aws:iam::${$AWS_ACCOUNT_ID}:role/Cognito_PollyUnauth_Role"



An error occurred (NotAuthorizedException) when calling the SetIdentityPoolRoles operation: Access to Role 'arn:aws:iam:::role/Cognito_PollyUnauth_Role' is forbidden.


Error: Command failed: SubmitCode: aws cognito-identity set-identity-pool-roles ` ...